# Service descriptions

This document contains a short introduction to [RDF](https://www.w3.org/TR/rdf11-primer/) using [rudof](https://rudof-project.github.io/). 


## Preliminaries: Install and configure rudof

The library is available as `pyrudof`.

In [1]:
!pip install pyrudof

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 23.2 MB/s eta 0:00:00


The main entry point if a class called `Rudof` through which most of the functionality is provided.

In [ ]:
from pyrudof import Rudof, RudofConfig, ServiceDescription

In order to initialize that class, it is possible to pass a RudofConfig instance which contains configuration parameters for customization.

In [3]:
rudof = Rudof(RudofConfig())

## Obtaining information about SPARQL endpoints

### When the SPARQL endpoint directly generates the service description in RDF (Uniprot)

It is possible to get information from SPARQL endpoints that publish [service descriptions](https://www.w3.org/TR/sparql11-service-description/) using the [VoID](https://www.w3.org/TR/void/) vocabulary.

In [ ]:
# @title

rudof.reset_all()
from pyrudof import ReaderMode, RDFFormat, ServiceDescriptionFormat, QueryResultFormat

In [67]:
service = "https://sparql.uniprot.org/sparql"

rudof.read_service_description(service, RDFFormat.Turtle, None, ReaderMode.Strict)


Now we can serialize the service description in JSON, for example, as:

In [68]:
service = rudof.get_service_description()
service_str = service.serialize(ServiceDescriptionFormat.Json)
print(f"Service description in JSON:\n{service_str}")


Service description in JSON:
{
  "title": "UniProt",
  "endpoint": "https://sparql.uniprot.org/sparql",
  "default_dataset": {
    "id": {
      "Iri": "https://sparql.uniprot.org/sparql#sparql-default-dataset"
    },
    "default_graph": {
      "id": {
        "Iri": "https://sparql.uniprot.org/.well-known/void#sparql-default-graph"
      },
      "triples": 239686443056
    },
    "named_graphs": [
      {
        "id": {
          "Iri": "http://sparql.uniprot.org/uniparc"
        },
        "name": "http://sparql.uniprot.org/uniparc",
        "graphs": [
          {
            "id": {
              "Iri": "https://sparql.uniprot.org/.well-known/void#_graph_uniparc!9e1e33f2"
            },
            "triples": 168372462292,
            "classes": 6,
            "property_partition": [
              {
                "id": {
                  "Iri": "https://sparql.uniprot.org/.well-known/void#uniparc!9845e14e!mentions"
                },
                "property": "http://schem

We also support the conversion from service descriptions to MIE format files:

In [69]:
mie = service.as_mie()
print(f"Service description in MIE format as YAML:\n{mie.as_yaml()}")

Service description in MIE format as YAML:
---
schema_info:
  title: UniProt
  endpoint: "https://sparql.uniprot.org/sparql"
prefixes:
  dcterms: "http://purl.org/dc/terms/"
  xsd: "http://www.w3.org/2001/XMLSchema#"
  void: "http://rdfs.org/ns/void#"
  pav: "http://purl.org/pav/"
  formats: "http://www.w3.org/ns/formats/"
  void_ext: "http://ldf.fi/void-ext#"
  "": "http://www.w3.org/ns/sparql-service-description#"
  rdf: "http://www.w3.org/1999/02/22-rdf-syntax-ns#"


### Using SPARQL CONSTRUCT queries

Some endpoints, like RDFPortal, have their service description located as a named graph which can be queried using a SPARQL CONSTRUCT query. In the case of RDFPortal, the service description is located in the graph [endpoint = "https://plod.dbcls.jp/repositories/RDFPortal_VoID"
](endpoint = "https://plod.dbcls.jp/repositories/RDFPortal_VoID"
).

In [70]:
endpoint = "https://plod.dbcls.jp/repositories/RDFPortal_VoID"


In [71]:
construct_query = """
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX sd: <http://www.w3.org/ns/sparql-service-description#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

CONSTRUCT WHERE {
  [
    a sd:Service ;
    sd:defaultDataset [
       a sd:Dataset ;
       sd:namedGraph [
         sd:name <http://sparql.uniprot.org/uniprot> ;
         a sd:NamedGraph ;
         sd:endpoint ?ep_url ;
         sd:graph [
           a void:Dataset ;
           void:triples ?total_count ;
           void:classes ?class_count ;
           void:properties ?property_count ;
           void:distinctObjects ?uniq_object_count ;
           void:distinctSubjects ?uniq_subject_count ;
           void:classPartition [
             void:class ?class_name ;
             void:entities ?class_triple_count
           ] ;
           void:propertyPartition [
             void:property ?property_name ;
             void:triples ?property_triple_count
           ]
         ]
       ]
     ]
  ] .
}
"""

It is possible to run SPARQL CONSTRUCT queries in rudof using:

In [72]:
rudof.add_endpoint(endpoint)

In [73]:
dbcls_turtle = rudof.run_query_construct_str(construct_query, QueryResultFormat.Turtle)



In [74]:
print(dbcls_turtle)

@prefix void: <http://rdfs.org/ns/void#> .
@prefix sd: <http://www.w3.org/ns/sparql-service-description#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdf4j: <http://rdf4j.org/schema/rdf4j#> .
@prefix sesame: <http://www.openrdf.org/schema/sesame#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix fn: <http://www.w3.org/2005/xpath-functions#> .

<http://rdfportal.org/ns/svcroot> a sd:Service;
  sd:defaultDataset <http://rdfportal.org/ns/root> .

<http://rdfportal.org/ns/root> a sd:Dataset;
  sd:namedGraph <http://rdfportal.org/ns/2c0544f688656cff12b9b69af828aaf3> .

<http://rdfportal.org/ns/2c0544f688656cff12b9b69af828aaf3> a sd:NamedGraph, void:Dataset;
  sd:name <http://sparql.uniprot.org/uniprot>;
  sd:endpoint <https://rdfportal.org/sib/sparql>;
  void:triples 53650641556;
  void:classes 109;
  void:properties 85;
  void:distinctObjects 102943

In [75]:
rudof.read_service_description_str(dbcls_turtle)

And now we can get the information of the service description as before with:

In [76]:
service = rudof.get_service_description()
service_str = service.serialize(ServiceDescriptionFormat.Json)
print(f"Service description in JSON:\n{service_str}")


Service description in JSON:
{
  "default_dataset": {
    "id": {
      "Iri": "http://rdfportal.org/ns/root"
    },
    "default_graph": null,
    "named_graphs": [
      {
        "id": {
          "Iri": "http://rdfportal.org/ns/2c0544f688656cff12b9b69af828aaf3"
        },
        "name": "http://sparql.uniprot.org/uniprot",
        "graphs": [
          {
            "id": {
              "Iri": "http://rdfportal.org/ns/2c0544f688656cff12b9b69af828aaf3"
            },
            "triples": 53650641556,
            "classes": 109,
            "property_partition": [
              {
                "id": {
                  "Iri": "http://rdfportal.org/ns/f6449ce38048ee49928d73f74dcb06db"
                },
                "property": "http://purl.uniprot.org/core/conflictingSequence",
                "triples": 348894
              },
              {
                "id": {
                  "Iri": "http://rdfportal.org/ns/e171acb5096ce05b7cc6f02e36c87523"
                },
      